In [25]:
import pandas as pd
import numpy as np
import csv
import random


In [15]:
#cd git/HSTFinalProject/Classifier-for-Hardware-Trojan-Detection/ROFreq/ROFreq/


0

In [17]:
# Read Excel Spreadsheet of
chip1 = pd.read_excel("/home/yujia/Documents/HW_Security/Project/ROFreq/Chip1.xlsx",header=None)

# Create array of Zeros
Zeros = np.zeros((25,),dtype = int)

# Create Numpy Target Vector
Target_np = Zeros
Target_np[0] = 1
Target_np[24]=1

# Create Pandas Series of Numpy Target Vector
Target = pd.Series(Target_np)

# Set Target Column of Dataframe to Target Series
chip1['Target'] = Target

In [19]:
chip1

,0,1,2,3,4,5,6,7,Target
0,189.2,201.2,223.2,234.4,220.8,226.4,220.0,249.2,1
1,177.2,188.0,208.8,219.2,206.4,212.0,205.2,232.8,0
2,171.2,181.6,201.6,211.6,199.6,204.8,198.8,224.8,0
3,167.2,177.6,197.2,206.8,195.2,200.4,194.0,220.0,0
4,163.6,174.0,193.2,202.8,191.2,196.4,190.0,215.6,0
5,160.4,170.0,188.8,198.4,186.8,191.6,185.6,210.4,0
6,155.6,165.2,183.6,192.8,181.6,186.4,180.8,204.8,0
7,152.0,161.2,179.6,188.0,177.6,182.0,176.4,200.0,0
8,168.4,178.8,198.4,208.0,196.4,201.6,195.2,221.6,0
9,183.6,194.8,216.4,226.8,213.6,219.6,212.8,241.6,0


In [169]:
xlsx_path = '/home/yujia/Documents/HW_Security/Project/ROFreq/Chip'
csv_path ='/home/yujia/Documents/HW_Security/Project/ROFreq_C/Chip'

for i in np.arange(1,34):
    data_xls = pd.read_excel('{}{}.xlsx'.format(xlsx_path,i), dtype=str, index_col=None)
    data_xls.to_csv('{}{}.csv'.format(csv_path,i), encoding='utf-8', index=False)

In [386]:
def csv_read(chip_num,type,csv_path):
    data= []
    data_list= []
    label_list = np.zeros(25)
    label_list[0] = 1
    label_list[-1] = 1
    with open('{}{}.csv'.format(csv_path,chip_num)) as csvfile:
        csv_reader = csv.reader(csvfile)
   
        for row in csv_reader:
            # rounding
            row = [round(float(num[0:12]),3) for num in row]     
            data_list.append(row)   

        if type == 'TI':
            data.append(data_list[0])
            data.append(data_list[-1])
            label = [1,1]
        elif type == 'TF':
            data = data_list[1:-1]
            label = label_list[1:-1]
        else:
            data = data_list
            label = label_list 
                  
        
    return data,label

In [389]:
def train_test_split(num_sel,case_sel,csv_path):
    chips_num = np.arange(1,34)
    indices = range(33)
    # randomly select #num chips
    sel = random.sample(indices,num_sel)
    # split tain and test data
    train_chips = chips_num[sel]
    test_chips = np.delete(chips_num,sel)
    print('Training chips num: ',train_chips)
    temp = []

    for i in range(int(num_sel/2)):
        temp.extend(['TF','TI'])
 
    type_sel_1 = random.sample(temp,num_sel)
    type_sel_3 = random.choices(['TI','TF'],k=num_sel)
    
    if case_sel == 1:
        print('Type selection:', type_sel_1)
        print('TI:',type_sel_1.count('TI'))
        print('TF:',type_sel_1.count('TF'))

    else:
        print('Type selection:', type_sel_3)
        print('TI:',type_sel_3.count('TI'))
        print('TF:',type_sel_3.count('TF'))

    cnt = 0
    train_data = []
    train_label = []

    for chip_num in train_chips:
        if case_sel == 1:
            data,label = csv_read(chip_num,type_sel_1[cnt],csv_path)
        elif case_sel ==3:
            data,label = csv_read(chip_num,type_sel_3[cnt],csv_path)
        train_data.extend(data)
        train_label.extend(label)
        cnt += 1

    test_data = []
    test_label = []
    for chip_num in test_chips:
        data,label = csv_read(chip_num,'ALL',csv_path)
        test_data.extend(data)
        test_label.extend(label)
    
    return train_data, train_label, test_data, test_label

In [390]:
# arg1 => number of training samples
# arg2 => number of case (enter 1 or 3 only)
train_data, train_label, test_data, test_label = train_test_split(24,3,csv_path)

Training chips num:  [ 2 30 13 16 28 32 31  7 11 12  5 25 21 20 15 22  3 10  8  9 26 29 27 18]
Type selection: ['TF', 'TI', 'TI', 'TI', 'TI', 'TF', 'TI', 'TF', 'TI', 'TI', 'TI', 'TI', 'TI', 'TI', 'TF', 'TI', 'TI', 'TF', 'TF', 'TF', 'TI', 'TF', 'TI', 'TF']
TI: 15
TF: 9
